# Text classification

## *"Words. I know words. I have the best words!"*
*- Noam Chomsky*

# Sentiment Analysis

<video controls src="cartoon.m4v" type="video/mp4" />

# Sentiment Analysis: Implementation

<video controls src="machine_learning.m4v" type="video/mp4" />

# Overview

In order to train a machine learning model to classify text, we need:
1. a way to preprocess text
2. a label for each text, represented as number
3. a way to represent each text as vector input
4. a model to learn  a function $f(input) = label$
5. a way to evaluate how well the model works
6. a way to predict new data

As an example, we will use reviews data and try to classify the rating into $positive$ or $negative$, only based on the text they use.

The same method can be used for any other data, including more labels and other dependent variables (e.g., age or gender of the text author, social constructs expressed in the text, etc...). 

# 1. Getting data

We use `pandas` to read in our data from a CSV file, but you can use almost any format (remember `read_excel()`, `read_sql()`, etc.)

In [ ]:
import pandas as pd

data = pd.read_csv('../data/sa_train.csv')
print(len(data), data['output'].unique())
data.head(2)

## Preprocessing

Text is messy. The goal of preprocessing is to reduce the amount of noise (= unnecessary variation), while maintaining the signal. There is no one-size-fits-all solution, but a good approximation is the following:

In [ ]:
import spacy
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
def clean_text(text):
    '''reduce text to lower-case lexicon entry'''
    lemmas = [token.lemma_ for token in nlp(text) 
              if token.pos_ in {'NOUN', 'VERB', 'ADJ', 'ADV', 'PROPN'}]
    return ' '.join(lemmas)

clean_text('This is a test sentence. And here comes another one... Go me!')

Let's clean up the input data. This can take a while, so it's good to save it... I have done that here already.

In [ ]:
# data['clean_text'] = data['input'].apply(clean_text)
data['clean_text']

# 2. Labels

Here, we assume that we already have the labels. (In your task, you will have to label them yourself! Hint: use `input()` or a spreadsheet).

However, in order for the machine learning model to work with the labels, we need to translate them into a vector of numbers. We can use `sklearn.LabelEncoder`

In [ ]:
from sklearn.preprocessing import LabelEncoder

# transform labels into numbers
labels2numbers = LabelEncoder()

y = labels2numbers.fit_transform(data['output'])
print(data['output'][:10], y[:10], len(y))

To get the original names back, use `inverse_transform()`:

In [ ]:
labels2numbers.inverse_transform([1,1,1,0,0,1])

# 3. Representing text

First, we need to transform the texts into a matrix, where each row represents one text instance. The columns are the **features**


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=0.001, max_df=0.75, stop_words='english')

X = vectorizer.fit_transform(data.clean_text)
print(X.shape)

We can now trasnlate back and forth between columns and words:

In [ ]:
vectorizer.vocabulary_['bad']

In [ ]:
vectorizer.get_feature_names()[4191]

In [ ]:
import numpy as np
# [vectorizer.get_feature_names()[f] for f in (X.sum(axis=1)).A1.argsort()[:10]]
vectorizer.get_feature_names()[1244]

Let's see how often that word is in the data:

In [ ]:
len(data[data.input.str.contains('bad')])

# 4. Learning a classification model

A classification model is simply a function that takes a text representation as input, and returns an output label.

Inside that function is normally a set of weights. By multiplying the weight vector with the input vector, we get the label.

## 4.1: Fitting a model

Fitting a model is the process of finding the right weights to map the training inputs to the training outputs. Fitting to data in `sklearn` is easy: we use the `fit()` function, giving it the input matrix and output vector.

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
%time classifier.fit(X, y)
print(classifier)

The resulting fitted model has coefficients (betas) for each word/feature in our vocabulary

In [ ]:
classifier.coef_.shape

We can now examine the weights/coefficients/betas for the individual words (note that each word has an ID):

In [ ]:
k = vectorizer.vocabulary_['bad'] # column position for the word
print(vectorizer.get_feature_names()[k], classifier.coef_[0, k])

NB: in a two-class problem, our coefficents are in a vector: positive values indicate the positive class, negative values the other class.
In a multi-class problem, we have one **row** of coefficients for each class: positive values indicate that this feature contributes to the class, negative values indicate that it contributes to other classes.

# 5. Evaluating models

Having a model is great, but how well does it do? Can it classify what it has seen? We need a way to estimate how well the model will work on new data.

We need a metric to measure performance and a way to simulate new data.

## 5.1: Metrics

We use three measure:
1. precision
2. recall
3. F1

### Precision

Precision measures how many of our model's predictions were correct. We divide the number of true positives by the number of all positives

$$
p = \frac{tp}{tp+fp}
$$

### Recall

Recall measures how many of the correct answers in the data our model managed to find. We divide the number of true positives by the number of true positives (the instances our model got) and false negatives (the instances our model *should* have gotten)

$$
r = \frac{tp}{tp+fn}
$$

### F1

A model that classified everything as, say, "positive" would get a perfect recall (it does, after all, find all positive examples). However, such a model would obviously be useless, since its precision is bad.

We want to balance the two against each other. F1 does exactly that, by taking the harmonic mean.

$$
F_1 = \frac{p\cdot r}{p+r}
$$

Luckily, all of these metrics are implemented in `sklearn`. All we have to provide are the predictions of our model, and the actual correct answers (called the *gold standard*). 

In [ ]:
from sklearn.metrics import classification_report

## 5.2: Cross-validation

How do we measure performance on new data, if we don't know what the correct outputs for those new data points are?

In **$k$-fold cross-validation**, we simulate new data, by fitting our model on parts of the data, and evaluating on other. We can thereby measure the performance on the held-out part. 

However, we have now reduced the amount of data we used to fit the data. In order to address this, we simply repeat the process $k$ times.
We separate the data into $k$ parts, fit the model on $k-1$ parts, and evaluate on the $k$th part. In the end, we have performance scores from $k$ models. The average of them tells us how well the model would work on new data.

![3-fold cross-validation](3foldCV.png)

In [ ]:
from sklearn.model_selection import cross_val_score

for k in [2,3,5,10]:
    cv = cross_val_score(LogisticRegression(), X, y=y, cv=k, n_jobs=-1, scoring="f1_micro")
    fold_size = X.shape[0]/k
    
    print("F1 with {} folds for bag-of-words is {}".format(k, cv.mean()))
    print("Training on {} instances/fold, testing on {}".format(fold_size*(k-1), fold_size))
    print()

## Baselines
So, is that performance good? Let's compare to a **baseline**, i.e., a null-hypothesis. The simplest one is that all instances belong to the most fereuqnt class in the data.

In [ ]:
from sklearn.dummy import DummyClassifier

most_frequent = DummyClassifier(strategy='most_frequent')

print(cross_val_score(most_frequent, X, y=y, cv=k, n_jobs=-1, scoring="f1_micro").mean())

# Activity

See whether you can apply the previous steps to a new data sets, a description of wines. Choose any of the descriptor columns as target variable. The text is already preprocessed, to save time.

In [ ]:
wine = pd.read_csv('../data/wine.csv', encoding='utf8')
wine

In [ ]:
# your code here

# 6 Classifying new data

Classifying new (**held-out**) data is called **prediction**. We reuse the weights we have learned before on a new data matrix to predict the new outcomes.
Important: the new data needs to have the same number of features!

In [ ]:
# read in new data set
new_data = pd.read_csv('../data/sa_test.csv')
print(len(new_data))
new_data.head()

Don't forget to clean it!

In [ ]:
%time new_data['clean_text'] = new_data.input.apply(clean_text)

Let's see how well we do on this data:

In [ ]:
# transform text into word counts
# IMPORTANT: use same vectorizer we fit on training data to create vectors!
new_X = vectorizer.transform(new_data.clean_text)

# use the old classifier to predict and evaluate
new_predictions = classifier.predict(new_X)
print(new_predictions)

Instead, we can also predict the probabilities of belonging to each class

In [ ]:
new_probabilities = classifier.predict_proba(new_X)
print(new_probabilities)

For each instance (=row), we get a probability distribution over the classes (=columns)

## 6.1 Regularization

Typically, performance is lower on unseen data, because our model **overfit** the training data: it expects the new data to look *exactly* the same as the training data. That is almost never true.

In order to prevent the model from overfitting, we need to **regularize** it. Essentially, we make it harder to learn the training data.

A simple example of regularization is to "corrupt" the training data by adding a little bit of noise to each training instance. Since the noise is irregular, it becomes harder for the model to learn any patterns.

In [ ]:
from scipy.sparse import random

num_instances, num_features = X.shape

for i in range(5):
    X_regularized = X + random(num_instances, num_features, density=0.01)

    print(cross_val_score(LogisticRegression(), X_regularized, y=y, cv=k, n_jobs=-1, scoring="f1_micro").mean())

If you run the previous cell several times, you see different results (it gets even more varied if you change `density`). This variation arises because we add **random** noise. Not good...

Instead, it makes sense to force the model to spread the weights more evenly over all features, rather than bet on a few feature, which mighht not be present in future data.

We can do this by training the model with the `C` parameter. The default is `1`. Lower values mean stricter regularization.

In [ ]:
from sklearn.metrics import f1_score
best_c = None
best_f1_score = 0.0
for c in [50, 20, 10, 1.0, 0.5, 0.1, 0.05, 0.01]:
    clf = LogisticRegression(C=c)
    cv_reg = cross_val_score(clf, X, y=y, cv=5, n_jobs=-1, scoring="f1_micro").mean()

    print("5-CV on train at C={}: {}".format(c, cv_reg.mean()))
    print()

    if cv_reg > best_f1_score:
        best_f1_score = cv_reg
        best_c = c
        
print("best C parameter: {}".format(best_c))

# Better features = better performance


We now have **a lot** of features! More than we have actual examples...

Not all of them will be helpful, though. Let's select the top 1500 based on how well they predict they outcome of the training data.

We use two libraries from `sklearn`, `SelectKBest` (the selection algorithm) and `chi2` (the selection criterion).

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=1500).fit(X, y)
X_sel = selector.transform(X)
print(X_sel.shape)

Let's see how well this new representation performs, by looking at the 5-fold cross-validation. We keep the best regularization value from before.

In [ ]:
clf = LogisticRegression(C=best_c)

cv_reg = cross_val_score(clf, X_sel, y=y, cv=5, n_jobs=-1, scoring="f1_micro")
print("5-CV on train: {}".format(cv_reg.mean()))

Not too bad! We have handily beaten our previous best! Let's fit a classifier on the whole data now.

In [ ]:
clf.fit(X_sel, y)

Now, let's apply it to the held-out data set. 
We need to 
* vectorize the data with our vectorizer from before (otherwise, we get different features)
* select the top features (using our previously fitted selector)

In [ ]:
# select features for new data
new_X_sel = selector.transform(new_X)
print(new_X_sel.shape)

Finally, we can use our new classifier to predict the new data labels, and compare them to the truth.

In [ ]:
new_predictions_regularized = clf.predict(new_X_sel)
prediction_df = pd.DataFrame(data={'input': new_data['input'], 'prediction': labels2numbers.inverse_transform(new_predictions_regularized), 'truth':new_data['output']})
prediction_df

In [ ]:
new_data['input'][9]

## Getting insights

In order to explore which features are most indicative, we need some code

In [ ]:
features = vectorizer.get_feature_names() # get the names of the features
top_scores = selector.scores_.argsort()[-1500:] # get the indices of the selection
best_indicator_terms = [features[i] for i in sorted(top_scores)] # sort feature names

top_indicator_scores = pd.DataFrame(data={'feature': best_indicator_terms, 'coefficient': clf.coef_[0]})
top_indicator_scores.sort_values('coefficient')

# Checklist: how to classify my data

1. label at ***least 2000*** tweets in your data set as `positive`, `negative`, or `neutral`
2. preprocess the text of *all* tweets in your data (labeled and unlabeled)
3. read in the labeled tweets and their labels
4. transform the labels into numbers
5. use `TfidfVectorizer` to extract the features and transform them into feature vectors
6. select the top $N$ features (where $N$ is smaller than the number of labeled tweets)
7. create a classifier
8. use 5-fold CV to find the best regularization parameter, top $N$ feature selection, and maybe feature generation and preprocessing steps

Once you are satisfied with the results:
9. read in the rest of the (unlabeled) tweets
10. use the `TfidfVectorizer` from 5. to transform the new data into vectors
11. use the `SelectKBest` selector from 6. to get the top $N$ features
12. use the classifier from 7. to predict the labels for the new data
13. save the predicted labels or probabilities to your database or an Excel file
